## Scraping from Wikipedia the Registration and Coordinates, if they exist.
https://en.wikipedia.org/wiki/List_of_accidents_and_incidents_involving_commercial_aircraft

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_accidents_and_incidents_involving_commercial_aircraft"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

# starting section  "1910s and 1920s"
section = soup.find("span", id="1910s_and_1920s").parent.parent

# Dataframe
df = pd.DataFrame(columns=["Registration", "Latitude", "Longitude"])

# Iterate all links that are in bold
for bold_link in section.find_all("b"):
    link = bold_link.find("a")
    if link:
        link = link["href"]
        sub_res = requests.get("https://en.wikipedia.org" + link)
        sub_soup = BeautifulSoup(sub_res.text, "html.parser")

        # Find the infobox
        infobox = sub_soup.find("table", class_="infobox")
        if infobox:
            # Need to check if the values wanted exist
            reg_col = infobox.find("th", text="Registration")
            lat_col = infobox.find("span", class_="latitude")
            lon_col = infobox.find("span", class_="longitude")
            if reg_col and lat_col and lon_col:
                # Extracting them
                reg = reg_col.find_next_sibling("td").text.strip()
                lat = lat_col.text.strip()
                lon = lon_col.text.strip()

                # Append the datframe
                df = df.append({"Registration": reg, "Latitude": lat, "Longitude": lon}, ignore_index=True)
df

,Registration,Latitude,Longitude
0,G-EAMA,51°34′13.5″N,0°12′11.07″W
1,F-GEAD,49°38′00″N,01°56′49″E
2,F-AEBY,49°42′36″N,2°10′19″E
3,F-AECB,51°16′16″N,0°25′41″E
4,G-EBBS,51°50′11″N,0°37′34″W
...,...,...,...
872,UR-CIC,40°58′N,24°12′E
873,HL7525,10°17′42″N,123°58′00″E
874,5H-PWF[1],01°20′07″S,31°49′33″E
875,CC-BHB[1],12°01′19″S,77°06′52″W


### some values contain unwanted strings

In [3]:
df['Registration'] = df['Registration'].str.split("[").str.get(0)
df['Registration'] = df['Registration'].str.split("(").str.get(0)
df

,Registration,Latitude,Longitude
0,G-EAMA,51°34′13.5″N,0°12′11.07″W
1,F-GEAD,49°38′00″N,01°56′49″E
2,F-AEBY,49°42′36″N,2°10′19″E
3,F-AECB,51°16′16″N,0°25′41″E
4,G-EBBS,51°50′11″N,0°37′34″W
...,...,...,...
872,UR-CIC,40°58′N,24°12′E
873,HL7525,10°17′42″N,123°58′00″E
874,5H-PWF,01°20′07″S,31°49′33″E
875,CC-BHB,12°01′19″S,77°06′52″W


### will use this file to concate with the planed crash info file

In [7]:
def convert_to_dd(dms):
    parts = re.findall(r'\d+', dms)
    if len(parts) < 2: #some coordinates do not have seconds
        return None
    d = int(parts[0])
    m = int(parts[1])
    s = 0
    if len(parts) == 3:
        s = float(parts[2])
    dd = d + m/60 + s/3600
    if dms[-1] == 'S' or dms[-1] == 'W':
        dd = dd * -1
    return dd

df['Latitude_DD'] = df['Latitude'].apply(lambda x: convert_to_dd(x))
df['Longitude_DD'] = df['Longitude'].apply(lambda x: convert_to_dd(x))

df

,Registration,Latitude,Longitude,Latitude_DD,Longitude_DD
0,G-EAMA,51°34′13.5″N,0°12′11.07″W,51.566667,-0.200000
1,F-GEAD,49°38′00″N,01°56′49″E,49.633333,1.946944
2,F-AEBY,49°42′36″N,2°10′19″E,49.710000,2.171944
3,F-AECB,51°16′16″N,0°25′41″E,51.271111,0.428056
4,G-EBBS,51°50′11″N,0°37′34″W,51.836389,-0.626111
...,...,...,...,...,...
872,UR-CIC,40°58′N,24°12′E,40.966667,24.200000
873,HL7525,10°17′42″N,123°58′00″E,10.295000,123.966667
874,5H-PWF,01°20′07″S,31°49′33″E,-1.335278,31.825833
875,CC-BHB,12°01′19″S,77°06′52″W,-12.021944,-77.114444


In [8]:
df.to_excel('Data/GeoLoc.xlsx', index=False)